In [1]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.0'
spark_version = 'spark-3.0.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:12 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Ign:13 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [20]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Software_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   42605767|R3EFW2STIYIY0I|B00MUTIDKI|     248732228|McAfee 2015 Inter...|        Software|          1|            2|          2|   N|                Y|I was very disapp...|I was very disapp...| 2015-08-31|
|         US|   51771800|R12NR0R5A9F7FT|B00EPACNUG|     531462352|Hallmark Card Stu...|        Software|          5|    

In [21]:
# Create the vine_table DataFrame
vine_df = df.select(['review_id', 'star_rating', 'helpful_votes', 'total_votes', 'vine', 'verified_purchase'])
vine_df.show(5)
vine_df.count()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3EFW2STIYIY0I|          1|            2|          2|   N|                Y|
|R12NR0R5A9F7FT|          5|            0|          0|   N|                Y|
|R1LSH74R9XAP59|          2|            0|          1|   N|                Y|
|R1QXUNTF76K7L6|          2|            0|          0|   N|                Y|
|R2F7DR75PS8NKT|          5|            0|          0|   N|                Y|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



341931

In [22]:
# Filter DataFrame to retrieve all rows where total_votes count is greater than or equal to 20
filtered_df = vine_df.filter("total_votes>=20").orderBy(vine_df["total_votes"].asc())
filtered_df.show()
filtered_df.count()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3C3UU3AWSPCPQ|          1|           13|         20|   N|                N|
|R1VZNSTRV9IE38|          1|           20|         20|   N|                Y|
|R2M9V2XMOA5MEJ|          1|           19|         20|   N|                Y|
|R1QMF01BDRT2LJ|          1|           10|         20|   N|                N|
|R2G908XLS4JISS|          5|           20|         20|   N|                N|
|R3E9P8SZEMFTBK|          2|           12|         20|   N|                N|
|R3K2ZVAQ2O3LWT|          1|            7|         20|   N|                Y|
|R1CXK4SGLJB4FE|          1|           15|         20|   N|                Y|
|R3GJGFTSDNVCHH|          5|           16|         20|   N|                N|
|R3JANDU5B4OSF0|          2|            6|         20|   N|     

20216

In [24]:
# Filter filtered_df to retrieve rows where helpful_votes/total_votes is greater than or equal to 50%
helpful_df = filtered_df.filter((filtered_df['helpful_votes']) / (filtered_df['total_votes']) >= 0.5)
helpful_df.show(5)
helpful_df.count()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1PXUULGJ2ARI8|          3|           20|         20|   N|                Y|
|R2G908XLS4JISS|          5|           20|         20|   N|                N|
|R3QDRWZTGQPEDB|          1|           18|         20|   N|                Y|
|R1QMF01BDRT2LJ|          1|           10|         20|   N|                N|
|R1CXK4SGLJB4FE|          1|           15|         20|   N|                Y|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



17762

# Vine Program Analysis

In [25]:
# Filter helpful_df to create table that only has the rows where a review was written as part of vine program(paid)
vine_program_df = helpful_df.filter(helpful_df['vine'] == 'Y')
vine_program_df.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1BGEGOX6SCYU2|          5|           19|         20|   Y|                N|
|R2SLQR7YTQELLA|          5|           19|         20|   Y|                N|
|R3BCPKHRR5942R|          3|           19|         20|   Y|                N|
|R3IZBBDG779H1G|          3|           17|         20|   Y|                N|
|R254QE8B67FICR|          5|           18|         20|   Y|                N|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



In [26]:
# Total number of vine program reviews
total_vine_reviews = vine_program_df.count()
print(f" There are a total of {total_vine_reviews} vine reviews")


 There are a total of 248 vine reviews


In [27]:
# 5-star reviews from vine program
five_star_vine_df = vine_program_df.filter(vine_program_df['star_rating'] == '5')

# 5-star review count from vine program
five_star_vine_reviews = five_star_vine_df.count()
print(f"There are {five_star_vine_reviews} five-star reviews from the vine program")

There are 102 five-star reviews from the vine program


In [28]:
# Percentage of 5-star reviews from vine program
percentage = round(((five_star_vine_reviews / total_vine_reviews) * 100), 0)
print(f"{percentage}% of vine reviews are five-star reviews")

41.0% of vine reviews are five-star reviews


# Non-Vine Program Analysis

In [29]:
# Filter helpful_df to create table that only has the rows where a review was not written as part of vine program(unpaid)
non_vine_program_df = helpful_df.filter(helpful_df['vine'] == 'N')
non_vine_program_df.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1PXUULGJ2ARI8|          3|           20|         20|   N|                Y|
|R2G908XLS4JISS|          5|           20|         20|   N|                N|
|R3QDRWZTGQPEDB|          1|           18|         20|   N|                Y|
|R1QMF01BDRT2LJ|          1|           10|         20|   N|                N|
|R1CXK4SGLJB4FE|          1|           15|         20|   N|                Y|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



In [30]:
# Total number of non-vine program reviews
total_non_vine_reviews = non_vine_program_df.count()
print(f"There are a total of {total_non_vine_reviews:,} non-vine reviews")

There are a total of 17,514 non-vine reviews


In [31]:
# 5-star reviews from non-vine program
five_star_non_vine_df = non_vine_program_df.filter(non_vine_program_df['star_rating'] == '5')

# 5-star review count from vine program
five_star_non_vine_reviews = five_star_non_vine_df.count()
print(f"There are {five_star_non_vine_reviews:,} five-star reviews that are non-vine reviews")

There are 5,154 five-star reviews that are non-vine reviews


In [32]:
# Percentage of 5-star reviews from non-vine program
percentage = round(((five_star_non_vine_reviews / total_non_vine_reviews) * 100), 0)
print(f"{percentage}% of non-vine reviews are five-star reviews")

29.0% of non-vine reviews are five-star reviews
